# **Governance Database Extraction and Preprocessing**

Governance Database Proj retrieves data directly from OPENDART to build a database of KOSPI-listed corporations and executive status available through posted disclosures. 

### Environment Set Up 
Runs on Python 3.13.5. The cell below will check that the current kernel is using the correct Python version and raise an error otherwise. To set a virtual environment, execute the following lines in the terminal: 

    python3.13.5 -m venv virtualenv

    virtualenv\Scripts\activate

In [4]:
import sys
assert sys.version_info >= (3, 13, 5)

### Outputs

By the end of the project, the following two databases will be produced: 
1. **executive_df**, providing details on the 15k+ listed executives, including information such as registered officer status, shareholder relations, salary, and professional experience.

2. **summary_df**,  a grouped dataset across corp-level information, including number of directory types, audit committee size, and total assets from the past three years (used to determine audit committee mandate).

*navigate to README.md file for reference*

<br> 
The cell below checks for the necessary folders. If it returns False, create a folder (at the same directory level as notebooks, not within) labeled 'data' and within it, two subfolders: 'raw' and 'processed'.

In [ ]:
import os

data_dir = os.path.join('..', 'data')
raw_dir = os.path.join(data_dir, 'raw')
processed_dir = os.path.join(data_dir, 'processed')

print(os.path.isdir(raw_dir) and os.path.isdir(processed_dir))

True


### **Data Extraction Notebook (data_extraction_ipynb)**

#### 0. save_df_to_csv
</b>

a helper function, called at the end of each function to save outputs as csv files within raw data folder

#### 1. get_corp_code
</b>


pulls the most up to date list of corp codes. Corp codes are unique reference codes assigned by OPENDART, distinct from stock number, and used as required keys to access and pull full company (2. get_kospi_company_info) and executive (3. get_executive_status_data) info.  
</b>

[OPENDART | Guide for Developers to Corporation code](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE001&apiId=AE00004)
</b>

Required Key: 
</b>

- crtfc_key (API key)

----

#### 2. get_kospi_company_info
</b>

passes in the list of corp codes from get_corp_code, filters for kospi codes, and appends all relevant company info. 

[OPENDART | Guide for Developers to Overview of corporate status](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE001&apiId=AE00002)

Required Keys: 
- crtfc_key (API key) 
- corp_code 

Kept Data: 
</b>


| Key  | Name | 
| -------|-----|
| corp_name | Formal name	  | 
| stock_code  | Stock item code	  | 
| ceo_nm  | Representative name  |
| induty_code*  |  Industry code   | 

*induty_code: not relevant now, could be used later to compare industry norms

Dropped Data: 
| Key  | Name |
| -------|-----|
| corp_name_eng  | English name	  | 
| stock_name | Item name 	  | 
| corp_cls  |  Corporation type   | 
| jurir_no  |  Corporate registration No.   |
| bizr_no | Business registration No.  | 
| adres | Address  | 
| hm_url  | Website URL  | 
| ir_url  | IR website  | 
| phn_no |  Telephone No.   | 
| fax_no  | Fax No.  | 
| est_dt  | Establishment date (YYYYMMDD)  | 
| acc_mt  | Month of settlement (MM)  |

corp code and name are sufficient for identification

----

#### 3. get_executive_status_data

retrieves executive-level data. 

[OPENDART | Guide for Developers to Status of executives](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE002&apiId=AE00011)

Required Keys:
- crtfc_key (API key)
- corp_code 
- bsns_year (fiscal year)
- reprt_code
    - First Quarterly Report : 11013
    - Semi-annual Report : 11012
    - Third Quarterly Report : 11014
    - Annual Report : 11011

Kept Data: 
| Key  | Name | 
| -------|-----|
| rcept_no | Filing No.  | 
| corp_cls | Corporation type	  |
| corp_code | Corporation code	  | 
| corp_name | Corporation name	  | 
| nm | Name  |
| sexdstn | Gender  | 
| ofcps | Position  | 
| rgist_exctv_at | Registered officer status  | 
| fte_at | Full-time  | 
| chrg_job | Responsibilites  | 
| main_career | Professional Background  |
| mxmm_shrholdr_relate | Relationship to Largest Shareholder  | 
| hffc_pd | Period of employment  | 

Dropped Data: 
| Key  | Name | 
| -------|-----|
| birth_ym | Date of birth  | 
| tenure_end_on | Term expiration date  | 
| stlm_dt | Settlement date  | 

tenure in company is sufficient for guaging expertise

stlm_dt irrelevant given filter for year and report type

----

#### 4. get_total_assets  

reads financial statement for corp to pull total assets from balance sheet. 

Assets will be used to check requirements for mandated audit committees (corporations with total assets > $2T KRW). Because corporations have a two year grace period for forming a mandated audit committee, the function pulls total assets from the past three years. 

[OPENDART | Single company’s full financial statements 개발가이드](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DE003&apiId=AE00036)

Required Keys:
- crtfc_key (API key)
- corp_code 
- bsns_year (fiscal year)
- reprt_code 
- fs_div (seperate/consolidated report)


For each corp, if a consolidated report exists, it will pull information from that statement. Otherwise, it will fall back on the seperate report. 
Supports pulling other FS data, so long as sj_div and sj_nm is specified. 

To see the full list of datapoints that can be extracted (ie Revenue, Profit, Liabilities)

Kept Data: 
| Key  | Name | 
| -------|-----|
| rcept_no* | Filing No.  | 
| thstrm_amount	| Term amount |
| frmtrm_amount	| Previous term amount | 
| bfefrmtrm_amount	| Amount of term before previous | 

Dropped Data: 
| Key  | Name | 
| -------|-----|
| reprt_code | Report code	  | 
| bsns_year | Fiscal year	  | 
| corp_code | Corporation code	  | 
| sj_div** | Type of financial statement	  |
| sj_nm | Financial statement title	  |
| account_id | Account ID  |
| account_nm | Account name  | 
| account_detail | Detail account  |
| thstrm_nm	| Term name  | 
| thstrm_add_amount	 | Accumulated term amount	  | 
| frmtrm_nm	| Previous term name | 
| frmtrm_q_nm | Previous term name(Quarterly/Semiannual) | 
| frmtrm_q_amount | Previous term amount(Quarterly/Semiannual) | 
| frmtrm_add_amount	| Accumulated previous term amount  | 
| bfefrmtrm_nm	| Name of term before previous | 
| ord	| Account code sort order | 
| currency	| Currency unit |

*required key for subdoc searches in preprocessing

**function already filters for sj_div = BS, can change to retrieve data from other statements 

----

#### 5. get_salary_type

pulls salary data from three OPDENDART source types: 
- Individual, which discloses the exact amount for executives making more than 500M KRW. 
- Grouped, which provides total annual grouped salary and average salaries by status type. 
- Unregistered, which provides the total annual grouped salary and average per person.

In the preprocessing notebook, salary will be appended to each executive - exact where possible and average amounts otherwise. 

1. [OPENDART | Guide for Developers to Remuneration for individual directors and auditors](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE002&apiId=AE00013) (lists all those > 500m KRW)

2. [OPENDART | Guide for Developers to Remuneration for all directors and auditors (remuneration paid - by type)](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE002&apiId=AE00030)

3. [OPENDART | Guide for Developers to Remuneration for unregistered executives](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE002&apiId=AE00028)

Required Keys:
- crtfc_key (API key)
- corp_code 
- bsns_year (fiscal year)
- reprt_code 

The resulting **salary_separate_df** displays the salary data pulled from the three datapoints, with standardized columns for the purposes of merging with **exec_df** in the preprocessing notebook. The following chart maps the resulting **salary_separate_df**'s column names to the corresponding OPENDART source.

| salary_separate_df column  | (1.) Individual | (2.) All By Type | (3.) Unregistered | 
| -------|-----|-----|-----|
| position | ofcps | se (category)* | se	(unregistered) |
| compensation | mendng_totamt | psn1_avrg_pymntamt | jan_salary_am |
| salary_source | 개인별보수 | 임원전체보수유형 | 미등기임원 |
| salary_type | exact | estimate | estimate | 


*Category covers:

- Registered director (excluding outside directors and members of the audit committee)
- Outside director (excluding members of the audit committee)
- Member of the audit committee
- Auditors

----


#### 6. get_salary_total  

pulls the total salary for each corp. 

[OPENDART | Guide for Developers to Remuneration for all directors and auditors](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE002&apiId=AE00014)

Required Keys:
- crtfc_key (API key)
- corp_code 
- bsns_year (fiscal year)
- reprt_code 

The resulting **salary_total_df** keeps all response variables. This includes: 
- nmpr (total headcount of all directors and auditors)
- mendng_totamt (total remuneration amount for all directors and auditors)
- jan_avrg_mendng_am (the average remuneration per person)

These points will be used to check **summary_df** values, to ensure that total headcount and remuneration totals align. The total remuneration amount is then merged with **summary_df** as a *Total Compensation* column.

-----

#### 7. get_major_shareholder_data

pulls holding status of major shareholders. 

In the preprocessing notebook, shareholder status will be merged with the exec data, such that if a registered or unregistered executive is listed as a major shareholder, their shares will be appended to exec_df.

[OPENDART | Guide for Developers to Information on largest shareholder](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE002&apiId=AE00008)

Required Keys:
- crtfc_key (API key)
- corp_code 
- bsns_year (fiscal year)
- reprt_code 

The resulting **major_shareholder_df** contains all response keys for potential further evaluation. When merged to **exec_df**, only *trmend_posesn_stock_qota_rt* (shareholding ratio at the end of the reporting period) is added as a *Shareholding Ratio* column. The following are not carried over: 
- bsis_posesn_stock_co	(number of stocks at the beginning of the reporting period)
- bsis_posesn_stock_qota_rt (shareholding ratio at the beginning of the reporting period)
- trmend_posesn_stock_co (number of stocks at the end of the reporting period)

Alternative Source: [OPENDART | Guide for Developers to Report of executives and major shareholders' ownership](https://engopendart.fss.or.kr/guide/detail.do?apiGrpCd=DE004&apiId=AE00041) which pulls stock transaction updates by executives and major shareholders
